# NLP con Long-Short Term Memory (LSTM)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion2/2-nlp-with-lstm.ipynb)

En este notebook implementaremos un clasificador de noticias en español utilizando la arquitectura de red LSTM. La idea es tener un punto de referencia para comparar cuando observemos la parte de transformers, por lo que utilizaremos el mismo dataset y tarea de ejemplo. Utilizarémos las utilidades de tokenización de huggingface transformers para ayudarnos con esta tarea.

#### Referencias
- Dataset: https://huggingface.co/datasets/MarcOrfilaCarreras/spanish-news
- [Long Short-Term Memory](https://www.researchgate.net/publication/13853244_Long_Short-Term_Memory#fullTextFileContent)

In [9]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

In [ ]:
#!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt && pip install -r requirements.txt
!test '{IN_COLAB}' = 'True' && sudo apt-get update -y
!test '{IN_COLAB}' = 'True' && sudo apt-get install python3.10 python3.10-distutils python3.10-lib2to3 -y
!test '{IN_COLAB}' = 'True' && sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.11 2
!test '{IN_COLAB}' = 'True' && sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 1
!test '{IN_COLAB}' = 'True' && pip install lightning datasets

### Cargando el dataset
Este es un dataset pequeño de articulos de noticias en idioma español con sus respectivas categorías. El dataset está disponible en el HuggingFace Hub y puede ser fácilmente descargado con la librería.

In [11]:
from datasets import load_dataset
import warnings
import os

warnings.filterwarnings("ignore")
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
dataset = load_dataset('MarcOrfilaCarreras/spanish-news', split='train')
dataset

Dataset({
    features: ['language', 'category', 'newspaper', 'hash', 'text'],
    num_rows: 10200
})

Observemos uno de sus registros

In [12]:
dataset[1]

{'language': 'es',
 'category': 'play',
 'newspaper': 'de_lector_a_lector',
 'hash': 'b387bc0a5ad68524c8aa5da489555ca41d5a3575',
 'text': 'El coraje de ser, de Mónica Cavallé, la aventura del autoconocimiento filosófico.Todos experimentamos momentos de plenitud vinculados a la expresión directa y auténtica de nosotros mismos: momentos de contemplación de la belleza del mundo en que nuestros sentidos se abren como si lo vieran por primera vez, de intimidad y comunión con otro ser humano, de fluidez creativa, de expresión confiada y libre… Estos momentos permiten intuir lo que puede ser una vida en la que no meramente se existe, sino en la que se vive en todo el sentido de esta palabra.Esta vida solo es posible cuando sabemos quiénes somos, cuando nos conocemos a nosotros mismos de modo experiencial: no cuando nos llenamos de ideas sobre nosotros, sino cuando nos asentamos en nuestro ser real, más allá de nuestras defensas, máscaras y falsos yoes.El coraje de ser es una invitación a aden

Para los efectos de esta tarea, nos servirán el texto y la categoría naturalmente.

A manera general, observemos que tan largos o cortos tienden a ser los textos.

In [13]:
text_lengths = [len(row['text']) for row in dataset]
print(f"Texto más corto: {min(text_lengths)}")
print(f"Texto más largo: {max(text_lengths)}")
print(f"Longitud promedio: {sum(text_lengths) / len(text_lengths)}")

Texto más corto: 501
Texto más largo: 204324
Longitud promedio: 4218.154509803921


Estos valores son la cantidad de *caractéres* que tiene las secuencias. Una decisión ingenua pero útil en este momento podría ser ajustar la longitud de las secuencias que vamos a usar para el entrenamiento a unos 2000 tokens. Esto podría ser suficiente para capturar una porción significativa de los textos.

## Definiendo el Tokenizer

Ahora, vamos a definir el tokenizer para nuestra tarea. Para mantener las cosas simples, vamos a mantener un conteo de palabras y vamos a hacer un corte hasta los primeros 50mil tokens.

In [ ]:
import re
from collections import Counter

def simple_tokenizer(text):
    text = text.lower()
    text = re.sub(r"[^a-záéíóúüñ]+", " ", text)
    return text.strip().split()

# Construimos el vocabulario a partir de conjunto de datos.
token_counts = Counter()
for text in dataset["text"]:
    token_counts.update(simple_tokenizer(text))

# 50k-2 porque necesitamos reservar espacio para los dos tokens especiales
top_n_tokens = list(token_counts.keys())[:50000-2]
vocab = {"[PAD]": 0, "[UNK]": 1}
for token in top_n_tokens:
    vocab[token] = len(vocab)

def tokenize_text(text, max_length=50):
    tokens = simple_tokenizer(text)
    ids = [vocab.get(tok, vocab["[UNK]"]) for tok in tokens[:max_length]]
    ids += [vocab["[PAD]"]] * (max_length - len(ids))
    return ids

Exploremos ahora el tokenizador obtenido.

In [15]:
print(f"Vocabulario: {len(vocab)} tokens")
print("Primeros 15 tokens:")
print(f"{top_n_tokens[:15]}")
print("15 tokens de en medio:")
print(f"{top_n_tokens[1000:1015]}")
print("Últimos 15 tokens:")
print(f"{top_n_tokens[-15:]}")

Vocabulario: 50000 tokens
Primeros 15 tokens:
['valladolid', 'misteriosa', 'es', 'el', 'título', 'del', 'nuevo', 'libro', 'que', 'acaba', 'de', 'publicar', 'la', 'editorial', 'almuzara']
15 tokens de en medio:
['trabajar', 'primero', 'seis', 'meses', 'guionista', 'antigua', 'city', 'tv', 'después', 'llamó', 'tres', 'medio', 'redactora', 'super', 'pop']
Últimos 15 tokens:
['risco', 'infecciosas', 'amalia', 'simoni', 'camagüey', 'instructor', 'proteinuria', 'disfunción', 'endotelial', 'diabéticos', 'amlodipine', 'bloqueador', 'aml', 'hctz', 'tolerada']


Esta forma de exploración es para darnos una idea de las palabras más utilizadas en el corpus y nos dará un indicio de si la tokenización es adecuada o no. Vemos que tenemos algunos stop words, como artículos (el, la) y conectores (del, que). Para una tarea de clasificación de texto podríamos prescindir de estos pero para facilitar las cosas y ya que los demás tokens lucen bien, podemos preservarlos.

Ahora veamos como convierte el tokenizador una oración muy sencilla:

In [16]:
tokenized = tokenize_text("hola mundo", max_length=8)
tokenized

[29340, 157, 0, 0, 0, 0, 0, 0]

Lo que obtenemos de vuelta son los ids de cada token según el vocabulario. Ahora algo importante que notamos aquí es el *padding*, durante el entrenamiento, queremos que las secuencias sean de tamaño fijo, para asi operar comodamente con matrices. Pero ya vimos que no todos los textos tienen la misma longitud. Entonces que hacer? para los que son más largos que una longitud dada simplemente cortamos, pero para los que son más cortos, debemos *rellenar* lo faltante con un *token especial de relleno o padding*. Y es justo lo que definimos allí, cuando la cadena es inferior a 8 **tokens**, entonces debemos hacer padding hasta que se cumplan los 8.

Si queremos saber a que token exactamente hacen referencia estos ids, simplemente revisamos el vocabulario que hemos construido:

In [17]:
id_2_token = {v: k for k, v in vocab.items()}
[id_2_token[token] for token in tokenized]

['hola', 'mundo', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']

Claramente vemos los 3 tokens como cadenas independientes (el padding se considera un token independiente).

### Definiendo el dataset de pytorch
Ahora podemos proceder a definir el dataset. Esto debería ser muy sencillo dado que nuestro dataset es pequeño y ya tenemos el tokenizador listo.

In [18]:
import torch
import numpy as np
from typing import Tuple, Dict
from torch.utils.data import Dataset

class SpanishNewsDataset(Dataset):

    def __init__(self, tokenizer, dataset, seq_length: int = 512):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.seq_length = seq_length
        # Definimos estos dos mapas para facilitarnos la tarea
        # de traducir de nombres de categoría a ids de categoría.
        self.id_2_class_map = dict(enumerate(np.unique(dataset[:]['category'])))
        self.class_2_id_map = {v: k for k, v in self.id_2_class_map.items()}
        self.num_classes = len(self.id_2_class_map)

    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        text, y = self.dataset[index]['text'], self.dataset[index]['category']
        y = self.class_2_id_map[y]
        data = {'input_ids': torch.tensor(self.tokenizer(text, max_length=self.seq_length))}
        data['y'] = torch.tensor(y)
        return data


    def __len__(self):
        return len(self.dataset)

Ahora instanciaremos el dataset entero. Para este experimento, definiremos un tamaño máximo de secuencia de 2048 **tokens**. Que según nuestra intuición arriba, debería ser suficiente para la tarea.

In [19]:
max_len = 512 
spanish_news_dataset = SpanishNewsDataset(tokenize_text, dataset, seq_length=max_len)
assert len(spanish_news_dataset) == len(dataset)

Y luego, procedemos a hacer el train-val-test split y crear los dataloaders.

In [20]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader

batch_size = 4 if not IN_COLAB else 12
train_dataset, val_dataset, test_dataset = random_split(spanish_news_dataset, lengths=[0.8, 0.1, 0.1])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)

## Definición del modelo LSTM

Ahora vamos a configurar un módulo pytorch simple para este problema. Vamos ha utilizar los embeddings, que vendrían siendo los vectores de palabra. Pytorch nos ofrece una capa con la que directamente podemos entrenarlos a partir de los token ids obtenidos. El resto consistirá en invocar una capa LSTM seguida de una capa densa para la clasificación.

Recordemos que las redes recurrentes como las LSTM por diseño enlazan todas las dimensiones del vector de entrada, formando así la secuencia, la estructura natural que necesitamos representar.

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTMBlock(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=2, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
    
    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, _) = self.lstm(embedded)
        return hidden[-1]


### Definición del clasificador

Finalmente, definimos el modelo en si. Este modelo constará de 3 capas:

- La tokenización, tal como la definimos anteriormente.
- El bloque LSTM, que acabamos de decinir.
- Una capa densa adicional que servirá como clasificador de aquello que nos entregue la capa del transformer.

Como este es un LightningModule, aquí definiremos el resto de funciones utilitarias para el entrenamiento de la tarea.

In [22]:
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchmetrics import Accuracy

class SpanishNewsClassifierWithLSTM(LightningModule):

    def __init__(self, vocab_size: int, num_classes: int, emb_dim: int, hidden_dim: int = 128):
        super(SpanishNewsClassifierWithLSTM, self).__init__()
        self.num_classes = num_classes
        self.lstm = LSTMBlock(vocab_size, emb_dim, hidden_dim, num_classes)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes),
            nn.LogSoftmax(dim=1)
        )

        self.train_acc = Accuracy(task='multiclass', num_classes=num_classes)
        self.val_acc = Accuracy(task='multiclass', num_classes=num_classes)
        self.test_acc = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        embeddings = self.lstm(x)
        return self.classifier(embeddings)

    
    def training_step(self, batch, batch_idx):
        x, y = batch['input_ids'], batch['y']
        # print(f"\nbatch-idx: {batch_idx}")
        # print(f"shape of x: {x.shape}")
        # print(torch.max(x, dim=0))
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.train_acc(y_hat, y)
        self.log('train-loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train-acc', self.train_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def validation_step(self, batch):
        x, y = batch['input_ids'], batch['y']
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.val_acc(y_hat, y)
        self.log('val-loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val-acc', self.val_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def test_step(self, batch):
        x, y = batch['input_ids'], batch['y']
        y_hat = self(x)
        self.test_acc(y_hat, y)
        self.log('test-acc', self.test_acc, prog_bar=True, on_step=False, on_epoch=True)


    def predict_step(self, batch):
        x = batch['input_ids']
        return self(x)


    def configure_optimizers(self):
        optimizer =  torch.optim.AdamW(self.parameters(), lr=1e-3, weight_decay=1e-5)
        return optimizer

    
model = SpanishNewsClassifierWithLSTM(vocab_size=len(vocab) + 1, num_classes=spanish_news_dataset.num_classes, emb_dim=256)

tb_logger = TensorBoardLogger('tb_logs', name='LSTMClassifier')
callbacks=[EarlyStopping(monitor='train-loss', patience=3, mode='min')]
trainer = Trainer(max_epochs=10, devices=1, logger=tb_logger, callbacks=callbacks, precision="16-mixed")

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | lstm       | LSTMBlock          | 13.1 M | train
1 | classifier | Sequential         | 200 K  | train
2 | train_acc  | MulticlassAccuracy | 0      | train
3 | val_acc    | MulticlassAccuracy | 0      | train
4 | test_acc   | MulticlassAccuracy | 0      | train
----------------------------------------------------------
13.3 M    Trainable params
0         Non-trainable params
13.3 M    Total params
53.322    Total estimated model params size (MB)
16        Modules in train

Epoch 9: 100%|██████████| 2040/2040 [00:25<00:00, 79.38it/s, v_num=0, val-loss=0.525, val-acc=0.907, train-loss=0.0621, train-acc=0.985]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 2040/2040 [00:26<00:00, 78.38it/s, v_num=0, val-loss=0.525, val-acc=0.907, train-loss=0.0621, train-acc=0.985]


Observemos el proceso de entrenamiento

In [23]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir tb_logs/

Y como es de esperarse, realizaremos la validación contra el conjunto de prueba.

In [25]:
model.eval()
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 255/255 [00:01<00:00, 237.12it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test-acc          │    0.9029411673545837     │
└───────────────────────────┴───────────────────────────┘

[{'test-acc': 0.9029411673545837}]

### Haciendo predicciones

Finalmente, vamos a hacer uso del modelo y ver que tan bueno es para la clasificación de noticias.

In [26]:
predictions = trainer.predict(model, test_loader)
predictions = torch.cat(predictions, dim=0)
predictions = torch.argmax(predictions, dim=-1)
predictions = [spanish_news_dataset.id_2_class_map[pred] for pred in predictions.numpy()]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 255/255 [00:01<00:00, 219.44it/s]


In [34]:
import pandas as pd

test_indices = test_dataset.indices
df = pd.DataFrame(data={
    "texto": dataset[test_indices]['text'],
    "tokens": [tokenize_text(v) for v in dataset[test_indices]['text']],
    "categoría": dataset[test_indices]['category'],
    'predicción': predictions
}, index=test_indices)

df['tokens_string'] = df.tokens.apply(lambda t: ' '.join([id_2_token[i] for i in t]))
df = df[["texto", "tokens", "tokens_string", "categoría", "predicción"]]
df.style.set_table_styles(
    [
        {'selector': 'td', 'props': [('word-wrap', 'break-word')]}
    ]
)
df.head(15)

,texto,tokens,tokens_string,categoría,predicción
6734,El Gobierno tiene un plan alternativo para ase...,"[5, 3270, 1058, 102, 1737, 4007, 240, 834, 10,...",el gobierno tiene un plan alternativo para ase...,politics,politics
1905,Prevalencia de hipotiroidismo subclínico y su ...,"[34012, 12, 1, 1, 36, 46, 703, 58, 1, 36, 1652...",prevalencia de [UNK] [UNK] y su relación con [...,medicine,medicine
3049,"El Sol, la fuente inagotable de energía que da...","[5, 14608, 14, 2248, 10396, 12, 2641, 10, 246,...",el sol la fuente inagotable de energía que da ...,astronomy,astronomy
1772,Mondor en la mama. A propósito de un casoLa en...,"[1, 54, 14, 1, 40, 554, 12, 102, 1, 1652, 12, ...",[UNK] en la [UNK] a propósito de un [UNK] enfe...,medicine,medicine
6980,Navantia acaba de firmar con la Marina de Noru...,"[1, 11, 12, 23970, 58, 14, 27045, 12, 27768, 1...",[UNK] acaba de firmar con la marina de noruega...,military,military
1589,Tienen una función parecida a los relojes o la...,"[440, 19, 2427, 14721, 40, 34, 14066, 96, 77, ...",tienen una función parecida a los relojes o la...,tech,tech
240,"En última instancia, cuando se apagan las luce...","[54, 1177, 5104, 206, 17, 16158, 77, 3867, 12,...",en última instancia cuando se apagan las luces...,play,play
7356,La compañía francesa Escape International ha p...,"[14, 1151, 2772, 7008, 29665, 260, 1048, 54, 1...",la compañía francesa escape international ha p...,military,military
6763,El Gobierno busca que Bruselas apruebe un impu...,"[5, 3270, 232, 10, 14032, 36279, 102, 15003, 2...",el gobierno busca que bruselas apruebe un impu...,politics,politics
3341,"comentarios 44Cybertron, el planeta de los Tra...","[7493, 1, 5, 3124, 12, 34, 8114, 5, 29619, 12,...",comentarios [UNK] el planeta de los transforme...,astronomy,astronomy


In [35]:
errors = df[df['categoría'] != df['predicción']]
errors.head(15)

,texto,tokens,tokens_string,categoría,predicción
1120,Pues parece que la policía holandesa se precip...,"[2020, 491, 10, 14, 1497, 38011, 17, 38012, 10...",pues parece que la policía holandesa se precip...,tech,play
7473,El nuevo Consejo de Administración de Fincanti...,"[5, 8, 17379, 12, 444, 12, 1, 1, 29649, 45, 14...",el nuevo consejo de administración de [UNK] [U...,military,economy
2765,"Fuente de la imagen, OtherJune Williams creció...","[2248, 12, 14, 564, 1, 10953, 5680, 57, 40, 34...",fuente de la imagen [UNK] williams creció junt...,astronomy,religion
4142,El Elche espera a Óscar Plano. El delantero ma...,"[5, 36649, 4040, 40, 2947, 1846, 5, 1, 8710, 3...",el elche espera a óscar plano el [UNK] madrile...,sport,economy
689,Algunos de los monumentos más famosos del Rein...,"[501, 12, 34, 23928, 130, 1018, 7, 2125, 8269,...",algunos de los monumentos más famosos del rein...,play,tech
5835,Hay anuncios que irremediablemente se quedan a...,"[124, 11554, 10, 12505, 17, 1895, 12251, 54, 2...",hay anuncios que irremediablemente se quedan a...,alimentation,astronomy
7368,"Navantia junto a otras 5 grandes empresas, Rep...","[1, 57, 40, 2255, 901, 8594, 1, 1, 14784, 3176...",[UNK] junto a otras grandes empresas [UNK] [UN...,military,motor
9001,La versión más accesible del Lucid Air llega p...,"[14, 2049, 130, 14266, 7, 1, 7550, 333, 45, 31...",la versión más accesible del [UNK] air llega p...,motor,economy
7846,Pedro Sánchez ha regalado toda una ley de adoc...,"[399, 9554, 260, 19502, 89, 19, 2565, 12, 1, 5...",pedro sánchez ha regalado toda una ley de [UNK...,religion,politics
8103,Es algo que está sorprendiendo en la red. Mill...,"[4, 559, 10, 482, 24341, 54, 14, 1063, 1059, 1...",es algo que está sorprendiendo en la red millo...,religion,astronomy


## Conclusiones

- En este caso tenemos una tarea de clasificación de texto de múltiples clases.
- Estamos usando un bloque LSTM como featurizer, es decir lo usamos para extraer features de las secuencias de entrada con las cuales harémos predicciones luego.
- Nótese que de las capas LSTM, solo nos interesa la última, ya que esta recupera todas las operaciones enalazadas anteriores.
- Observamos que el modelo toma su tiempo en entrenar, esto es natural debido al diseño de las LSTM, donde por cada paso de tiempo se debe computar un gradiente, por lo que el computo es mucho mayor.